In [1]:
import sys
sys.path.append('../')
from hidden import MONGO_USR, MONGO_PWD
from pymongo import MongoClient
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# Database connection
client = MongoClient('mongodb://'+ MONGO_USR +':'+ MONGO_PWD +'@127.0.0.1:27017/?authSource=admin')
db = client['movielens']
movies = db['movies']
users = db['users']

In [2]:
# Retrieve datas from the database
movies = pd.DataFrame(movies.find({}, {"_id": 1, "title": 1}))
users = list(users.find({}, {"movies.movieid": 1, "_id": 1, "movies.rating": 1, "movies.timestamp": 1}))

# Convert users dataset into a DataFrame
users = pd.json_normalize(users)

# Explode the list of movies
users = users.explode('movies')

# Convert every element of the list into a distinct column
users_tmp = users['movies'].apply(pd.Series)
users = pd.concat([users, users_tmp], axis=1).drop('movies', axis=1)

print('taille de movies :', len(movies))
print('taille de users :', len(users))
users.head()

taille de movies : 3883
taille de users : 1000209


,_id,movieid,rating,timestamp
0,6040,573,4,956704056
0,6040,589,4,956704996
0,6040,1,3,957717358
0,6040,2068,4,997453982
0,6040,592,2,956716016


In [3]:
merged_df = movies.merge(users, left_on='_id', right_on='movieid')

# drop "movieid" column
merged_df = merged_df.drop(columns=['movieid'])

# rename "_id_x" column to "movieid"
merged_df = merged_df.rename(columns={"_id_x": "movie_id"})

# rename "_id_y" column to "user_id"
merged_df = merged_df.rename(columns={"_id_y": "user_id"})

# Sort merged_df by timestamp
merged_df = merged_df.sort_values(by=['timestamp'])

# Reset index
merged_df = merged_df.reset_index(drop=True)

# Drop the first entire line because of the odd size of the dataset
merged_df = merged_df.drop(merged_df.index[0])

print('Taille de merged_df :', len(merged_df))
merged_df.tail(10)


Taille de merged_df : 1000208


,movie_id,title,user_id,rating,timestamp
1000199,3098,"Natural, The (1984)",5948,4,1046437932
1000200,3267,"Mariachi, El (1992)",5312,4,1046444711
1000201,2453,"Boy Who Could Fly, The (1986)",4958,4,1046454260
1000202,2043,Darby O'Gill and the Little People (1959),4958,1,1046454282
1000203,3489,Hook (1991),4958,4,1046454320
1000204,2399,Santa Claus: The Movie (1985),4958,1,1046454338
1000205,1407,Scream (1996),4958,5,1046454443
1000206,3264,Buffy the Vampire Slayer (1992),4958,4,1046454548
1000207,2634,"Mummy, The (1959)",4958,3,1046454548
1000208,1924,Plan 9 from Outer Space (1958),4958,4,1046454590


In [4]:
# Number of ratings per movie
movies_counts = merged_df['movie_id'].value_counts()
print(movies_counts.describe())

print('\n')

# Number of ratings per user
cusers_counts = merged_df['user_id'].value_counts()
print(cusers_counts.describe())

# Define the thresholds under which we drop the movies
movies_threshold = 33
users_threshold = 44

# Drop movies with less than 33 ratings
merged_df = merged_df[merged_df['movie_id'].isin(movies_counts[movies_counts > movies_threshold].index)]

# Drop users with less than 44 ratings
merged_df = merged_df[merged_df['user_id'].isin(cusers_counts[cusers_counts > users_threshold].index)]

print('\n')
print('Nouvelle taille de merged_df :', len(merged_df))

count    3706.000000
mean      269.888829
std       384.046465
min         1.000000
25%        33.000000
50%       123.500000
75%       350.000000
max      3428.000000
Name: movie_id, dtype: float64


count    6040.000000
mean      165.597351
std       192.746879
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: user_id, dtype: float64


Nouvelle taille de merged_df : 942215


In [6]:
# Save merged_df to a csv file
merged_df.to_csv('data/merged_df.csv', index=False)